In [ ]:
%%shell

cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF


apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg


cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

apt-get update
apt-get install chromium chromium-driver

pip install -q selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [ ]:
# ブラウザをheadlessモード実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行。コラボの場合、必須。
options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')

In [ ]:
#インスタンス化
driver = webdriver.Chrome('chromedriver',options=options)
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)

#urlの指定
url="https://app.tmhub.jp/sign_in"

driver.get(url)
time.sleep(2)

print("サイトのタイトル：", driver.title)

In [ ]:
html = driver.page_source
BeautifulSoup(html,"lxml")

In [ ]:
login = "kkytmkmmkcrucial@gmail.com"
password = "hoge"

# ユーザー名入力
driver.find_element(By.NAME,"email").send_keys(login)

# パスワード入力
driver.find_element(By.NAME,"password").send_keys(password)

#ログインボタンクリック
btn = driver.find_element(By.XPATH,'//*[@id="root"]/div/div/div/div/div/div[2]/div[2]/div/div[1]/form/div[3]/button')
btn.click()

time.sleep(2)
html = driver.page_source
BeautifulSoup(html,"lxml")

In [ ]:
#ログイン後，メンバーページに遷移
url2="https://app.tmhub.jp/members"

driver.get(url2)
time.sleep(2)

print("サイトのタイトル：", driver.title)

In [ ]:
#メンバー一覧を表示
html = driver.page_source
soup=BeautifulSoup(html,"lxml")
##　要変数化自動取得
mem_list=soup.find_all(class_="MemberList_MemberListOpen__xA9ot")
print(mem_list)

In [ ]:
import re
#login者
own='matsuo'
#メンバーidを上のdivから取得
target='id="[0-9]{7}'
idx=re.findall(target,str(mem_list))
avg_list=list()
avg_list.clear
for i in range(len(idx)):
  url3="https://app.tmhub.jp/members/"+str(idx[i][4:11])
  driver.get(url3)
  time.sleep(3)
  html = driver.page_source
  soup=BeautifulSoup(html,"lxml")
  #要変数化&自動取得
  name=soup.find_all(class_="MemberDetailView_MemberDetailBox__TjOHm")
  name_head=str(name).find('<dl class="MemberDetailView_MemberDetailBox__TjOHm"><dt>')
  name_end=str(name).find('</dt><dd>Number:')
  #loginした人だけ，html表記が異なる？
  if own  in str(name):
    name_end=str(name).find('<div class="MemberDetailView_EditButton')


  per_score=soup.find_all("li")
  target='AVG</span></div><div class="listValue">.*<.'
  per_score=re.findall(target,str(per_score))
  avg_head=str(per_score).find('AVG</span></div><div class="listValue">')
  avg_end=str(per_score).find('</div></li>, <li class="listItem"><div class="listKey"><span class="listkey">RBI')
  #名前,id,打率のリスト

  avg_list.append([str(name)[name_head+56:name_end],str(idx[i][4:11]),str(per_score)[avg_head+39:avg_end]])


print(avg_list)

In [ ]:
 avg_list.sort(reverse=True, key=lambda x:x[2])
 print(avg_list[0][0])




In [ ]:
import csv

csv_path = "/content/drive/MyDrive/"

csv_name = 'personal_score.csv'
for i in range(len(avg_list)):
    del avg_list[i][1]
print(avg_list)
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(avg_list)